# Cubic spline interpolation with PyDynamic.uncertainty.interpolate.interp1d_unc

Interpolate a non-equidistant sine signal using cubic / bspline method with uncertainty propagation. Comparing the resulting uncertainties to a Monte-Carlo experiment yields good overlap. 

## Imports

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from PyDynamic.uncertainty.interpolate import interp1d_unc

## Create non-equidistant sine-signal

In [ ]:
# base signal
N = 10
t = np.cumsum(0.5 * (1 + np.random.random(N)))
x = np.sin(t) # + 0.01*np.random.randn(N)
#ut = np.full_like(t, 0.05)
ux = np.full_like(x, 0.2)

## Interpolate with PyDynamic

In [ ]:
ti = np.linspace(np.min(t), np.max(t), 60)
ti, xi, uxi = interp1d_unc(ti, t, x, ux, kind="cubic")

## Interpolate with Monte Carlo

In [ ]:
X_mc = []
for i in range(2000):
    interp_x = interp1d(t, x + ux * np.random.randn(len(x)), kind="cubic")
    xm = interp_x(ti)
    X_mc.append(xm)
x_mc = np.mean(X_mc, axis=0)
ux_mc = np.std(X_mc, axis=0)

## Compare results

### Visual

In [ ]:
# interpolated signal
plt.plot(ti, xi, '-or', label="interpolation PyDynamic")
plt.fill_between(ti, xi + uxi, xi - uxi, color="r", alpha=0.3)
# interpolated signal
plt.plot(ti, x_mc, '-ok', label="interpolation Monte Carlo")
plt.fill_between(ti, x_mc + ux_mc, x_mc - ux_mc, color="k", alpha=0.3)
# original signal
plt.plot(t, x, '-ob', label="original")
plt.fill_between(t, x + ux, x - ux, color="b", alpha=0.3)

plt.legend()

### Numerical

In [ ]:
np.allclose(xi, x_mc, atol=1e-2, rtol=1e-2)

In [ ]:
np.allclose(uxi, ux_mc, atol=1e-2, rtol=1e-2)